In [1]:
# Imports
import os
from os.path import join as join_path
import numpy as np
rng_seed = 399
np.random.seed(rng_seed)
import pickle
import pandas as pd
from tqdm.auto import tqdm
from nltk.tokenize import sent_tokenize

import tensorflow as tf
tf.random.set_seed(rng_seed)
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.sequence import skipgrams, make_sampling_table
from tensorflow.keras.utils import plot_model, to_categorical, Progbar

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

import multiprocessing
num_cores = multiprocessing.cpu_count()

In [2]:
# Define constants
# ----------------
data_dir = 'data'
checkpoints_dir = 'checkpoints'
text8_data_sentences_path = join_path(data_dir, 'fil9-sents.p')
text8_w2v_words_path = join_path(data_dir, 'fil9-w2v-words.p')
os.makedirs(checkpoints_dir, exist_ok=True)

# Training constants
sampling_window_size = 5
num_negative_samples = 15
min_word_count = 5
embedding_dim = 300
epochs = 20
# ----------------

# Load data

In [3]:
# Load text8 data sequences from data preprocessing
with open(text8_data_sentences_path, 'rb') as file:
    text8_sents = pickle.load(file)

In [4]:
# Create data iterator for Gensim
class Text8IteratorWord2Vec():
    def __init__(self, sentences: list):
        self.sentences = sentences
    
    def __iter__(self):
        for sent in self.sentences:
            yield sent.split()
text8_data_iter = Text8IteratorWord2Vec(text8_sents)

## Fit Word2vec model

In [5]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save models weights after each epoch.'''

    def __init__(self, output_dir: str, prefix: str, start_epoch: int = 1):
        self.output_dir = output_dir
        self.prefix = prefix
        self.epoch = start_epoch

    def on_epoch_end(self, model):
        output_path = join_path(self.output_dir, f'{self.prefix}-epoch-{self.epoch}.weights')
        
        # Save word embeddings among with words to file
        with open(output_path, 'wb') as file:
            pickle.dump(model.wv.vectors, file)

        self.epoch += 1

In [6]:
train_model = True
if train_model:
    w2v_model = Word2Vec(
        sentences=text8_data_iter,
        size=embedding_dim,
        workers=num_cores - 1,
        negative=num_negative_samples,
        window=sampling_window_size,
        min_count=min_word_count,
        iter=epochs,
        callbacks=[EpochSaver(checkpoints_dir, 'word2vec-model')]
    )

2020-05-26 17:42:28,969 : INFO : collecting all words and their counts
2020-05-26 17:42:28,970 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-26 17:42:29,018 : INFO : PROGRESS: at sentence #10000, processed 149792 words, keeping 19346 word types
2020-05-26 17:42:29,061 : INFO : PROGRESS: at sentence #20000, processed 297220 words, keeping 29172 word types
2020-05-26 17:42:29,110 : INFO : PROGRESS: at sentence #30000, processed 472759 words, keeping 39258 word types
2020-05-26 17:42:29,319 : INFO : PROGRESS: at sentence #40000, processed 631556 words, keeping 47987 word types
2020-05-26 17:42:29,368 : INFO : PROGRESS: at sentence #50000, processed 798591 words, keeping 55345 word types
2020-05-26 17:42:29,413 : INFO : PROGRESS: at sentence #60000, processed 964880 words, keeping 60511 word types
2020-05-26 17:42:29,458 : INFO : PROGRESS: at sentence #70000, processed 1125938 words, keeping 66687 word types
2020-05-26 17:42:29,505 : INFO : PROGRESS: at

2020-05-26 17:42:32,442 : INFO : PROGRESS: at sentence #710000, processed 11346614 words, keeping 250297 word types
2020-05-26 17:42:32,486 : INFO : PROGRESS: at sentence #720000, processed 11507605 words, keeping 252364 word types
2020-05-26 17:42:32,532 : INFO : PROGRESS: at sentence #730000, processed 11677644 words, keeping 254368 word types
2020-05-26 17:42:32,579 : INFO : PROGRESS: at sentence #740000, processed 11846267 words, keeping 256667 word types
2020-05-26 17:42:32,628 : INFO : PROGRESS: at sentence #750000, processed 12008816 words, keeping 258856 word types
2020-05-26 17:42:32,672 : INFO : PROGRESS: at sentence #760000, processed 12160772 words, keeping 260379 word types
2020-05-26 17:42:32,716 : INFO : PROGRESS: at sentence #770000, processed 12312650 words, keeping 262318 word types
2020-05-26 17:42:32,763 : INFO : PROGRESS: at sentence #780000, processed 12467475 words, keeping 264347 word types
2020-05-26 17:42:32,806 : INFO : PROGRESS: at sentence #790000, processe

2020-05-26 17:42:35,766 : INFO : PROGRESS: at sentence #1420000, processed 22843180 words, keeping 375690 word types
2020-05-26 17:42:35,811 : INFO : PROGRESS: at sentence #1430000, processed 22995319 words, keeping 377564 word types
2020-05-26 17:42:35,857 : INFO : PROGRESS: at sentence #1440000, processed 23148440 words, keeping 379038 word types
2020-05-26 17:42:35,903 : INFO : PROGRESS: at sentence #1450000, processed 23305195 words, keeping 380739 word types
2020-05-26 17:42:35,948 : INFO : PROGRESS: at sentence #1460000, processed 23449182 words, keeping 382492 word types
2020-05-26 17:42:35,994 : INFO : PROGRESS: at sentence #1470000, processed 23602338 words, keeping 384851 word types
2020-05-26 17:42:36,041 : INFO : PROGRESS: at sentence #1480000, processed 23761096 words, keeping 386419 word types
2020-05-26 17:42:36,089 : INFO : PROGRESS: at sentence #1490000, processed 23919878 words, keeping 388114 word types
2020-05-26 17:42:36,133 : INFO : PROGRESS: at sentence #1500000,

2020-05-26 17:42:38,967 : INFO : PROGRESS: at sentence #2130000, processed 33762494 words, keeping 494664 word types
2020-05-26 17:42:39,006 : INFO : PROGRESS: at sentence #2140000, processed 33896700 words, keeping 495782 word types
2020-05-26 17:42:39,037 : INFO : PROGRESS: at sentence #2150000, processed 34006700 words, keeping 496733 word types
2020-05-26 17:42:39,064 : INFO : PROGRESS: at sentence #2160000, processed 34103264 words, keeping 497358 word types
2020-05-26 17:42:39,095 : INFO : PROGRESS: at sentence #2170000, processed 34215948 words, keeping 498120 word types
2020-05-26 17:42:39,131 : INFO : PROGRESS: at sentence #2180000, processed 34347221 words, keeping 499139 word types
2020-05-26 17:42:39,155 : INFO : PROGRESS: at sentence #2190000, processed 34433436 words, keeping 499524 word types
2020-05-26 17:42:39,183 : INFO : PROGRESS: at sentence #2200000, processed 34540879 words, keeping 500378 word types
2020-05-26 17:42:39,211 : INFO : PROGRESS: at sentence #2210000,

2020-05-26 17:42:40,739 : INFO : PROGRESS: at sentence #2840000, processed 40077274 words, keeping 528995 word types
2020-05-26 17:42:40,756 : INFO : PROGRESS: at sentence #2850000, processed 40141188 words, keeping 529040 word types
2020-05-26 17:42:40,774 : INFO : PROGRESS: at sentence #2860000, processed 40205547 words, keeping 529100 word types
2020-05-26 17:42:40,793 : INFO : PROGRESS: at sentence #2870000, processed 40269749 words, keeping 529128 word types
2020-05-26 17:42:40,812 : INFO : PROGRESS: at sentence #2880000, processed 40335583 words, keeping 529188 word types
2020-05-26 17:42:40,830 : INFO : PROGRESS: at sentence #2890000, processed 40398734 words, keeping 529219 word types
2020-05-26 17:42:40,849 : INFO : PROGRESS: at sentence #2900000, processed 40465078 words, keeping 529276 word types
2020-05-26 17:42:40,867 : INFO : PROGRESS: at sentence #2910000, processed 40530580 words, keeping 529317 word types
2020-05-26 17:42:40,886 : INFO : PROGRESS: at sentence #2920000,

...

2020-05-26 18:08:32,370 : INFO : EPOCH 17 - PROGRESS: at 82.83% examples, 722552 words/s, in_qsize 28, out_qsize 1
2020-05-26 18:08:33,370 : INFO : EPOCH 17 - PROGRESS: at 83.67% examples, 723276 words/s, in_qsize 25, out_qsize 4
2020-05-26 18:08:34,371 : INFO : EPOCH 17 - PROGRESS: at 84.62% examples, 724864 words/s, in_qsize 29, out_qsize 3
2020-05-26 18:08:35,435 : INFO : EPOCH 17 - PROGRESS: at 85.32% examples, 723621 words/s, in_qsize 28, out_qsize 4
2020-05-26 18:08:36,435 : INFO : EPOCH 17 - PROGRESS: at 86.17% examples, 724556 words/s, in_qsize 26, out_qsize 3
2020-05-26 18:08:37,447 : INFO : EPOCH 17 - PROGRESS: at 87.15% examples, 726416 words/s, in_qsize 28, out_qsize 1
2020-05-26 18:08:38,459 : INFO : EPOCH 17 - PROGRESS: at 87.88% examples, 725427 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:08:39,459 : INFO : EPOCH 17 - PROGRESS: at 88.78% examples, 726525 words/s, in_qsize 29, out_qsize 1
2020-05-26 18:08:40,460 : INFO : EPOCH 17 - PROGRESS: at 89.70% examples, 727545

2020-05-26 18:09:32,905 : INFO : EPOCH 18 - PROGRESS: at 39.13% examples, 755522 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:09:33,946 : INFO : EPOCH 18 - PROGRESS: at 41.11% examples, 750152 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:09:34,955 : INFO : EPOCH 18 - PROGRESS: at 43.48% examples, 746244 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:09:35,960 : INFO : EPOCH 18 - PROGRESS: at 45.66% examples, 743321 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:09:37,042 : INFO : EPOCH 18 - PROGRESS: at 48.12% examples, 738363 words/s, in_qsize 30, out_qsize 1
2020-05-26 18:09:38,047 : INFO : EPOCH 18 - PROGRESS: at 50.59% examples, 735266 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:09:39,056 : INFO : EPOCH 18 - PROGRESS: at 53.35% examples, 732365 words/s, in_qsize 22, out_qsize 4
2020-05-26 18:09:40,087 : INFO : EPOCH 18 - PROGRESS: at 55.46% examples, 728285 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:09:41,092 : INFO : EPOCH 18 - PROGRESS: at 57.41% examples, 725400

2020-05-26 18:10:34,343 : INFO : EPOCH 19 - PROGRESS: at 11.97% examples, 758894 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:10:35,396 : INFO : EPOCH 19 - PROGRESS: at 12.94% examples, 760620 words/s, in_qsize 29, out_qsize 7
2020-05-26 18:10:36,451 : INFO : EPOCH 19 - PROGRESS: at 13.78% examples, 754993 words/s, in_qsize 30, out_qsize 0
2020-05-26 18:10:37,462 : INFO : EPOCH 19 - PROGRESS: at 14.80% examples, 761770 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:10:38,467 : INFO : EPOCH 19 - PROGRESS: at 15.76% examples, 764839 words/s, in_qsize 26, out_qsize 3
2020-05-26 18:10:39,507 : INFO : EPOCH 19 - PROGRESS: at 16.58% examples, 758843 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:10:40,511 : INFO : EPOCH 19 - PROGRESS: at 17.56% examples, 764036 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:10:41,528 : INFO : EPOCH 19 - PROGRESS: at 18.30% examples, 765935 words/s, in_qsize 27, out_qsize 2
2020-05-26 18:10:42,724 : INFO : EPOCH 19 - PROGRESS: at 19.20% examples, 759237

2020-05-26 18:11:48,370 : INFO : EPOCH 19 - PROGRESS: at 98.08% examples, 722679 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:11:49,379 : INFO : EPOCH 19 - PROGRESS: at 99.04% examples, 723871 words/s, in_qsize 28, out_qsize 1
2020-05-26 18:11:50,387 : INFO : EPOCH 19 - PROGRESS: at 99.80% examples, 723128 words/s, in_qsize 21, out_qsize 0
2020-05-26 18:11:50,432 : INFO : worker thread finished; awaiting finish of 14 more threads
2020-05-26 18:11:50,445 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-05-26 18:11:50,465 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-05-26 18:11:50,470 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-05-26 18:11:50,478 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-05-26 18:11:50,481 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-05-26 18:11:50,487 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-05-26 18

2020-05-26 18:12:49,543 : INFO : EPOCH 20 - PROGRESS: at 72.62% examples, 706001 words/s, in_qsize 28, out_qsize 10
2020-05-26 18:12:50,549 : INFO : EPOCH 20 - PROGRESS: at 73.66% examples, 709231 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:12:51,555 : INFO : EPOCH 20 - PROGRESS: at 74.57% examples, 711142 words/s, in_qsize 28, out_qsize 1
2020-05-26 18:12:52,576 : INFO : EPOCH 20 - PROGRESS: at 75.22% examples, 709397 words/s, in_qsize 26, out_qsize 5
2020-05-26 18:12:53,578 : INFO : EPOCH 20 - PROGRESS: at 76.21% examples, 711511 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:12:54,582 : INFO : EPOCH 20 - PROGRESS: at 77.12% examples, 713123 words/s, in_qsize 30, out_qsize 1
2020-05-26 18:12:55,598 : INFO : EPOCH 20 - PROGRESS: at 77.86% examples, 711739 words/s, in_qsize 29, out_qsize 0
2020-05-26 18:12:56,599 : INFO : EPOCH 20 - PROGRESS: at 78.78% examples, 713392 words/s, in_qsize 30, out_qsize 0
2020-05-26 18:12:57,627 : INFO : EPOCH 20 - PROGRESS: at 79.73% examples, 71504

In [7]:
# Save words used in Word2vec model to file
with open(text8_w2v_words_path, 'wb') as file:
    pickle.dump(np.array(w2v_model.wv.index2word), file)